In [1]:
import librosa
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
# Load Saved Dataset
train_data_value = np.load('saved_dataset/40_2048_512_train_data_value.npy')
train_data_target = np.load('saved_dataset/40_2048_512_train_data_target.npy')
test_data_value = np.load('saved_dataset/40_2048_512_test_data_value.npy')
test_data_target = np.load('saved_dataset/40_2048_512_test_data_target.npy')

## Data Exploration And Pre-processing

In [2]:
# SIgnal
explore_folder = os.listdir("./dataset/inspect/")
signal_list = {"Name": [], "Signal": []}
for x in explore_folder:
    signal, sample_rate = librosa.load("./dataset/inspect/{}".format(x), sr=16000)
    signal_list["Name"].append(x)
    signal_list["Signal"].append(signal)

# MfCC Constant
SAMPLE_RATE = 16000
n_mfcc = 40
n_fft = 2048
hop_length=512

# MfCC
mfcc_list = {"Name": [], "Vector": []}
for x,y in zip(signal_list["Name"],signal_list["Signal"]):
    mfcc = librosa.feature.mfcc(
    y=y, sr=SAMPLE_RATE, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc_list["Name"].append(x)
    mfcc_list["Vector"].append(mfcc)    

# MFCC Padded
padded_mfcc_list = {"Name": [], "Vector": []}
for x,y in zip(mfcc_list["Name"],mfcc_list["Vector"]):
    padded_data = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=156, padding="post")
    padded_mfcc_list["Name"].append(x)
    padded_mfcc_list["Vector"].append(padded_data)    

In [10]:
# Plot to Graph Function
def plot_to_waveform(name, signal, type):
    plt.figure(figsize=(4,2))
    plt.title(name, size=8)
    plt.xlabel('Time (s)', fontsize=8)
    plt.xticks(fontsize=8)
    plt.ylabel('Amplitude', fontsize=8)
    plt.yticks(fontsize=8)
    librosa.display.waveshow(signal,sr=SAMPLE_RATE,color='pink')
    if type == "save":
        plt.savefig("gambar_data/signal/{}.png".format(name[:-4]))
    else:
        return plt.show()
    
def plot_to_mfcc(name, signal, type):
    plt.figure(figsize=(4,2))
    plt.title(name, size=8)
    plt.xlabel('Frame', fontsize=8)
    plt.xticks(fontsize=8)
    plt.ylabel('MFCC', fontsize=8)
    plt.yticks(fontsize=8)
    librosa.display.specshow(signal,sr=SAMPLE_RATE, n_fft=n_fft, hop_length=hop_length )
    plt.gca().set_ylabel('MFCC Coefficients', labelpad=10)
    plt.gca().set_xlabel('Frame', labelpad=10)
    plt.colorbar(format="%+2.f dB")

    plt.tight_layout()
    if type == "save":
        plt.savefig("gambar_data/mfcc/{}.png".format(name[:-4]))
    elif type == "save_padded":
        plt.savefig("gambar_data/mfcc_padded/{}.png".format(name[:-4]))
    else:
        return plt.show()

In [8]:
# plot_to_mfcc(mfcc_list["Name"][1], padded_data, "show")
# for x,y in zip(signal_list["Name"],signal_list["Signal"]):
#     plot_to_waveform(x, y)
# for x,y in zip(mfcc_list["Name"],mfcc_list["Vector"]):
#     plot_to_mfcc(x, y, "save")
# for x,y in zip(padded_mfcc_list["Name"],padded_mfcc_list["Vector"]):
#     plot_to_mfcc(x, y, "save_padded")

## Model Resnet

In [3]:
def resnet_model_1():
    model = tf.keras.Sequential()
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))
    model.add(base_model)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
    # model.add(tf.keras.layers.Dense(1024, activation='relu'))
    # model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.summary()
    csv_logger = tf.keras.callbacks.CSVLogger('resnet_unweighted.csv')
    # history = model.fit(train_data_value, train_data_target, validation_data=(
    #     test_data_value, test_data_target), batch_size=64, epochs=30, callbacks=[csv_logger])
    return model

In [4]:
modelasdqw = resnet_model_1()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 5, 2, 2048)        23558528  
                                                                 
 time_distributed (TimeDistr  (None, 5, 4096)          0         
 ibuted)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 256)              4326400   
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 1542      
                                                                 
Total params: 27,886,470
Trainable params: 27,841,030
Non-trainable params: 45,440
_________________________________________________________________


In [9]:
def resnet_lstm_unweighted(optimizer='adam', learning_rate=0.0001):
    model = tf.keras.Sequential()
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))
    model.add(base_model)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.get(optimizer)
    optimiser.learning_rate.assign(learning_rate)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    str_learning_rate = str(learning_rate).replace('.', '')
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=5)
    csv_logger = tf.keras.callbacks.CSVLogger(
        'resnet_lstm{0}_{1}.csv'.format(optimizer, str_learning_rate))
    # model.summary()
    base_model.summary()
    # history = model.fit(train_data_value, train_data_target, validation_data=(
    #     test_data_value, test_data_target), batch_size=32, epochs=90, callbacks=[csv_logger, early_stop])
    return model

In [8]:
modelasdqwz = resnet_lstm_unweighted()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 5, 2, 2048)        23558528  
                                                                 
 time_distributed_2 (TimeDis  (None, 5, 4096)          0         
 tributed)                                                       
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              4326400   
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 27,886,470
Trainable params: 27,841,030
Non-trainable params: 45,440
_________________________________________________________________


In [10]:
modelasdqwzz = resnet_lstm_unweighted()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 156, 40, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 162, 46, 1)   0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 78, 20, 64)   3200        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 80, 22, 64)   0           ['conv1_conv[0][0]']    

In [13]:
def resnet_model_weighted():
    train_data_value_reshaped = np.zeros((train_data_value.shape[0], train_data_value.shape[1], train_data_value.shape[2], 3))
    train_data_value_reshaped[..., 0] = train_data_value
    train_data_value_reshaped[..., 1] = train_data_value
    train_data_value_reshaped[..., 2] = train_data_value
    test_data_value_reshaped = np.zeros((test_data_value.shape[0], test_data_value.shape[1], test_data_value.shape[2], 3))
    test_data_value_reshaped[..., 0] = test_data_value
    test_data_value_reshaped[..., 1] = test_data_value
    test_data_value_reshaped[..., 2] = test_data_value
    model = tf.keras.Sequential()
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False, weights="imagenet", input_shape=(train_data_value_reshaped.shape[1], train_data_value_reshaped.shape[2], 3))
    for layer in base_model.layers:
        layer.trainable = False
    model.add(base_model)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    csv_logger = tf.keras.callbacks.CSVLogger('resnet_weighted.csv')
    history = model.fit(train_data_value_reshaped, train_data_target, validation_data=(
        test_data_value_reshaped, test_data_target), batch_size=64, epochs=30, callbacks=[csv_logger])
    return model, history

In [3]:
def resnet_model_weighted_trainable():
    train_data_value_reshaped = np.zeros((train_data_value.shape[0], train_data_value.shape[1], train_data_value.shape[2], 3))
    train_data_value_reshaped[..., 0] = train_data_value
    train_data_value_reshaped[..., 1] = train_data_value
    train_data_value_reshaped[..., 2] = train_data_value
    test_data_value_reshaped = np.zeros((test_data_value.shape[0], test_data_value.shape[1], test_data_value.shape[2], 3))
    test_data_value_reshaped[..., 0] = test_data_value
    test_data_value_reshaped[..., 1] = test_data_value
    test_data_value_reshaped[..., 2] = test_data_value
    model = tf.keras.Sequential()
    base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False, weights="imagenet", input_shape=(train_data_value_reshaped.shape[1], train_data_value_reshaped.shape[2], 3))
    model.add(base_model)
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    csv_logger = tf.keras.callbacks.CSVLogger('resnet_weighted_trainable.csv')
    history = model.fit(train_data_value_reshaped, train_data_target, validation_data=(
        test_data_value_reshaped, test_data_target), batch_size=64, epochs=30, callbacks=[csv_logger])
    return model, history

In [14]:
resnet_model_weighted, resnet_weighted_history = resnet_model_weighted()

94668760/94668760 [==============================] - 27s 0us/step
Epoch 1/30
94/94 [==============================] - 18s 113ms/step - loss: 1.7493 - accuracy: 0.2498 - val_loss: 1.7700 - val_accuracy: 0.2317
Epoch 2/30
94/94 [==============================] - 8s 86ms/step - loss: 1.7463 - accuracy: 0.2476 - val_loss: 1.7292 - val_accuracy: 0.2565
Epoch 3/30
94/94 [==============================] - 8s 87ms/step - loss: 1.7161 - accuracy: 0.2699 - val_loss: 1.7046 - val_accuracy: 0.2760
Epoch 4/30
94/94 [==============================] - 8s 86ms/step - loss: 1.7039 - accuracy: 0.2738 - val_loss: 1.6669 - val_accuracy: 0.2989
Epoch 5/30
94/94 [==============================] - 8s 86ms/step - loss: 1.6888 - accuracy: 0.2728 - val_loss: 1.6564 - val_accuracy: 0.3015
Epoch 6/30
94/94 [==============================] - 8s 86ms/step - loss: 1.6658 - accuracy: 0.3004 - val_loss: 1.6605 - val_accuracy: 0.2968
Epoch 7/30
94/94 [==============================] - 8s 86ms/step - loss: 1.6637 - accu

In [4]:
resnet_model_weighted_trainable, resnet_weighted_trainable_history = resnet_model_weighted_trainable()
resnet_model_weighted_trainable.save("resnet_model_trainable_weight")


Epoch 1/30
94/94 [==============================] - 37s 263ms/step - loss: 1.4502 - accuracy: 0.4090 - val_loss: 2.0030 - val_accuracy: 0.3304
Epoch 2/30
94/94 [==============================] - 21s 221ms/step - loss: 1.3239 - accuracy: 0.4732 - val_loss: 2.4130 - val_accuracy: 0.2807
Epoch 3/30
94/94 [==============================] - 21s 222ms/step - loss: 1.2393 - accuracy: 0.5110 - val_loss: 2.5449 - val_accuracy: 0.3264
Epoch 4/30
94/94 [==============================] - 21s 222ms/step - loss: 1.1645 - accuracy: 0.5548 - val_loss: 2.3452 - val_accuracy: 0.3237
Epoch 5/30
94/94 [==============================] - 20s 217ms/step - loss: 1.1332 - accuracy: 0.5631 - val_loss: 2.5571 - val_accuracy: 0.2942
Epoch 6/30
94/94 [==============================] - 21s 220ms/step - loss: 1.0692 - accuracy: 0.5935 - val_loss: 1.8947 - val_accuracy: 0.3660
Epoch 7/30
94/94 [==============================] - 21s 223ms/step - loss: 1.0230 - accuracy: 0.6135 - val_loss: 1.6168 - val_accuracy: 0.4768

INFO:tensorflow:Assets written to: resnet_model_trainable_weight\assets


INFO:tensorflow:Assets written to: resnet_model_trainable_weight\assets


In [7]:
resnet_model_unweighted, resnet_unweighted_history = resnet_model_1()
resnet_model_unweighted.save("resnet_model_unweighted")

Epoch 1/30
94/94 [==============================] - 32s 252ms/step - loss: 1.5680 - accuracy: 0.3430 - val_loss: 3.0238 - val_accuracy: 0.1706
Epoch 2/30
94/94 [==============================] - 21s 225ms/step - loss: 1.4434 - accuracy: 0.4070 - val_loss: 3.1265 - val_accuracy: 0.1706
Epoch 3/30
94/94 [==============================] - 21s 222ms/step - loss: 1.3606 - accuracy: 0.4572 - val_loss: 3.7027 - val_accuracy: 0.1706
Epoch 4/30
94/94 [==============================] - 21s 224ms/step - loss: 1.3887 - accuracy: 0.4393 - val_loss: 2.7338 - val_accuracy: 0.2445
Epoch 5/30
94/94 [==============================] - 21s 223ms/step - loss: 1.3273 - accuracy: 0.4720 - val_loss: 2.4987 - val_accuracy: 0.3674
Epoch 6/30
94/94 [==============================] - 21s 225ms/step - loss: 1.2750 - accuracy: 0.5004 - val_loss: 1.5170 - val_accuracy: 0.3714
Epoch 7/30
94/94 [==============================] - 21s 221ms/step - loss: 1.3947 - accuracy: 0.4368 - val_loss: 1.9422 - val_accuracy: 0.3150

INFO:tensorflow:Assets written to: resnet_model_unweighted\assets


INFO:tensorflow:Assets written to: resnet_model_unweighted\assets


In [19]:
resnet_model_weighted.save("resnet_model_weighted")

INFO:tensorflow:Assets written to: resnet_model_weighted\assets


INFO:tensorflow:Assets written to: resnet_model_weighted\assets


## Model CNN-LSTM

In [3]:
def cnn_lstm():
    model = tf.keras.Sequential()
    # base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    #     include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
    train_data_value.shape[1], train_data_value.shape[2], 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))


    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.summary()
    # csv_logger = tf.keras.callbacks.CSVLogger('robust_rmsprop.csv')
    # history = model.fit(train_data_value, train_data_target, validation_data=(
    #     test_data_value, test_data_target), batch_size=32, epochs=30, callbacks=[csv_logger])
    return model

In [57]:
newmodel = cnn_lstm()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 154, 38, 64)       640       
                                                                 
 batch_normalization_48 (Bat  (None, 154, 38, 64)      256       
 chNormalization)                                                
                                                                 
 activation_48 (Activation)  (None, 154, 38, 64)       0         
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 77, 19, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_55 (Conv2D)          (None, 75, 17, 64)        36928     
                                                                 
 batch_normalization_49 (Bat  (None, 75, 17, 64)     

In [53]:
newmodel = cnn_lstm()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 154, 38, 64)       640       
                                                                 
 batch_normalization_40 (Bat  (None, 154, 38, 64)      256       
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 154, 38, 64)       0         
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 77, 19, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 75, 17, 64)        36928     
                                                                 
 batch_normalization_41 (Bat  (None, 75, 17, 64)     

In [4]:
newmodexxl = cnn_lstm()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 154, 38, 64)       1792      
                                                                 
 activation (Activation)     (None, 154, 38, 64)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 77, 19, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 17, 64)        36928     
                                                                 
 activation_1 (Activation)   (None, 75, 17, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 9, 64)        0         
 2D)                                                    

In [43]:
xx = 'adam'
yy = 0.02
str_learning_rate = str(yy).replace('.','')
print('robust_cnn_lstm{}{}.csv'.format(xx, str_learning_rate))

robust_cnn_lstmadam002.csv


In [15]:
def cnn_lstmx():
    model = tf.keras.Sequential()
    # base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    #     include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
    train_data_value.shape[1], train_data_value.shape[2], 1)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))


    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.LSTM(128))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.summary()
    # csv_logger = tf.keras.callbacks.CSVLogger('resnet_unweighted.csv')
    # history = model.fit(train_data_value, train_data_target, validation_data=(
    #     test_data_value, test_data_target), batch_size=32, epochs=60, callbacks=[csv_logger])
    return model
xxxxx = cnn_lstmx()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 154, 38, 64)       640       
                                                                 
 batch_normalization_32 (Bat  (None, 154, 38, 64)      256       
 chNormalization)                                                
                                                                 
 activation_32 (Activation)  (None, 154, 38, 64)       0         
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 77, 19, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 75, 17, 64)        36928     
                                                                 
 batch_normalization_33 (Bat  (None, 75, 17, 64)      

In [51]:
optimizer_list = ['adam','x','a']
learning_rate_list = [0.001, 0.01]

for x in list(optimizer_list):
    for y in learning_rate_list:
        print(x)

adam
adam
x
x
a
a


In [7]:
def cnnx():
    model = tf.keras.Sequential()
    # base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    #     include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
    train_data_value.shape[1], train_data_value.shape[2], 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))


    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.summary()
    # csv_logger = tf.keras.callbacks.CSVLogger('robust_rmsprop.csv')
    # history = model.fit(train_data_value, train_data_target, validation_data=(
    #     test_data_value, test_data_target), batch_size=32, epochs=30, callbacks=[csv_logger])
    return model

In [8]:
modelcnn = cnnx()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 154, 38, 64)       1792      
                                                                 
 batch_normalization_4 (Batc  (None, 154, 38, 64)      256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 154, 38, 64)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 77, 19, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 75, 17, 64)        36928     
                                                                 
 batch_normalization_5 (Batc  (None, 75, 17, 64)      